# Examples of Sampling Algorithms

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import torch
import torch.distributions as D
import seaborn as sns
import pandas as pd
import numpy as np

from .mcmc import visualize_sampling_1d, visualize_sampling_2d
from .mcmc import *

## Inverse transform sampling

We can use the inverse cumulative density function of a target distribution to convert samples from the uniform distribution into samples from the target distribution. This is a very general technique for producing samples from any distribution with an inverse CDF.

Since the CDF has range $(0, 1)$, we effectively sample uniformly in the range, and convert to the corresponding value in the domain. Let the CDF of some target distribution $p(z)$ be denoted by $f(\cdot)$. Then

1. Draw $u\sim \mathcal{U}(0, 1)$

2. Transform $z = f^{-1}(u)$ to obtain a sample $z\sim p(z)$.

In [ ]:
target_dist = D.Normal(torch.Tensor([0]), torch.Tensor([1]))

In [ ]:
samples = inverse_transform_sampling(target_dist.icdf, n_samples=1000)

In [ ]:
visualize_sampling_1d(samples, target_dist)

## Rejection sampling
https://en.wikipedia.org/wiki/Rejection_sampling#Adaptive_rejection_sampling

In [ ]:
#target_dist = D.Uniform(0, 1)
target_dist = D.Normal(1.0, 0.1)

In [ ]:
#proposal_dist = D.Independent(D.Normal(torch.tensor([0.5]), torch.tensor([0.5])), 1)
proposal_dist = D.Normal(0.5, 0.5)
proposal_dist.batch_shape

In [ ]:
df = pd.DataFrame({'Target distribution': target_dist.sample((10000,)),
                   'Proposal distribution': proposal_dist.sample((10000,)).flatten()})
ax = sns.displot(df, kde=True, stat='density')

In [ ]:
if hasattr(target_dist.support, 'lower_bound') and hasattr(target_dist.support, 'upper_bound'):
    val = torch.linspace(target_dist.support.lower_bound, target_dist.support.upper_bound, 500)
else:
    val = torch.linspace(-3, 3, 500)
M = (target_dist.log_prob(val) - proposal_dist.log_prob(val)).max().exp()
print(M)

In [ ]:
samples, estimands = rejection_sampling(n_samples=1000, proposal_dist=proposal_dist,
                                        target_dist_log_prob=target_dist.log_prob, M=M)
estimands

# Markov Chain Monte Carlo Samplers

TODO 
- Add evaluation metrics (R_hat, correlation between chains etc.) as in BDA 3
- Maybe make classes for the sampling methods and a base-class with the common evaluation metrics

In [ ]:
#class MCMCSampler():
#    def __init__(self):

## Gibbs sampling

Gibbs sampling is also called "alternating conditional sampling"

## Metropolis

Metropolis sampling samples from a target distribution $p(x)$, which is assumed difficult to sample from but easy to evaluate, by instead drawing samples from a (symmetric) proposal distribution $q(x)$, which is easy to sample from. Proposed samples are then accepted or rejected based on the probability ratio of the previous accepted sample to the new sample as evaluated under the target distribution (or a function that is proportional to it. $f(x) \propto p(x)$).

Let $f(x)$ be a function that is proportional to the desired probability distribution $p(x)$ (a.k.a. a target distribution).

- Initialization:
    - Choose an arbitrary point $x_t$ to be the first sample
    - Choose an arbitrary probability density $g(x\mid y)$ (sometimes written $Q(x\mid y)$) that suggests a candidate for the next sample value $x$, given the previous sample value $y$. For Metropolis sampling, $g$ is assumed to be symmetric; in other words, it must satisfy $g(x\mid y) = g(y\mid x)$. A usual choice is to let $g(x\mid y)$ be a Gaussian distribution centered at $y$, so that points closer to $y$ are more likely to be visited next, making the sequence of samples into a random walk.  The function $g$ is referred to as the **proposal density** or **jumping distribution**.
- For each iteration **t**:
    - **Generate** a candidate $x'$ for the next sample by picking from the distribution $g(x'\mid x_t)$.
    - **Calculate** the **acceptance ratio** $\alpha = f(x')/f(x_t)$, which will be used to decide whether to accept or reject the candidate. Because **f** is proportional to the density of **P**, we have that $\alpha = f(x')/f(x_t) = P(x')/P(x_t)$.
    - **Accept or reject**: 
        - Generate a uniform random number $u \in [0, 1]$.
        - If $u \le \alpha$, then **accept** the candidate by setting $x_{t+1} = x'$,
        - If $u > \alpha$, then **reject** the candidate and set $x_{t+1} = x_t$ instead.


In [ ]:
mix = D.Categorical(torch.ones(2,))
comp = D.Independent(D.Normal(torch.Tensor([[0, 0],[4, 6]]), torch.Tensor([[1, 1], [1, 2]])), 1)
target_dist = D.MixtureSameFamily(mix, comp)

In [ ]:
proposal_dist = D.MultivariateNormal(loc=torch.Tensor([0,0]), covariance_matrix=4 * torch.Tensor([[1, 0], [0, 1]]))

In [ ]:
initial_point = torch.Tensor([0, 10])
n_samples = 1000
samples, accept_reject = metropolis(initial_point, n_samples, proposal_dist, target_dist.log_prob)
torch.tensor(accept_reject).mean()

In [ ]:
visualize_sampling_2d(samples, target_dist)

## Metropolis-Hastings
https://en.wikipedia.org/wiki/Metropolis–Hastings_algorithm

The Metropolis algorithm requires a symmetric proposal distribution. The Metropolis-Hastings algorithm relaxes this requirement by computing the likelihood ratio of the proposals drawn under the proposal distribution and correcting the likelihood ratio of the proposals under the target distribution. This effectively corrects for the effects of an asymmetric proposal distribution.

In [ ]:
mix = D.Categorical(torch.ones(2,))
comp = D.Independent(D.Normal(torch.Tensor([[0, 0],[4, 6]]), torch.Tensor([[1, 1], [1, 2]])), 1)
target_dist = D.MixtureSameFamily(mix, comp)

In [ ]:
# Mixture of zero mean small variance gaussian and zero mean high variance gaussian
mix = D.Categorical(torch.ones(2,))
comp = D.Independent(D.Normal(torch.Tensor([[0, 0],[0, 0]]), torch.Tensor([[0.5, 0.5], [5, 5]])), 1)
proposal_dist = D.MixtureSameFamily(mix, comp)

#proposal_dist = D.Independent(D.LogNormal(torch.tensor([1.0, 1.0]), torch.tensor([1.0, 1.0])), 1)

In [ ]:
initial_point = torch.tensor([0.1, 5.0])
n_samples = 3000
samples, accept_reject = metropolis_hastings(initial_point, n_samples, proposal_dist, target_dist.log_prob)
torch.tensor(accept_reject).mean()

In [ ]:
visualize_sampling_2d(samples[n_samples // 2:], target_dist)

### Batched Metropolis Hastings

In [ ]:
mix = D.Categorical(torch.ones(2,))
comp = D.Independent(D.Normal(torch.Tensor([[0, 0],[4, 6]]), torch.Tensor([[1, 1], [1, 2]])), 1)
target_dist = D.MixtureSameFamily(mix, comp)

In [ ]:
n_chains = 10

In [ ]:
proposal_dist = D.Independent(D.Normal(torch.tensor([0.0, 0.0]), torch.tensor([2.5, 3.5])), 1)
proposal_dist.batch_shape, proposal_dist.event_shape

In [ ]:
initial_point = torch.tensor([2, 2.5]) + 1.5 * torch.randn(size=(n_chains, 2,)) 
n_samples = 1000
samples, estimands = metropolis_hastings_batched(initial_point, n_samples, proposal_dist, target_dist.log_prob)

In [ ]:
metrics, variogram, autocorrelations, T = convergence_test(estimands[0], estimands[1][:n_samples // 2])
metrics

In [ ]:
data = {name: estimands[1][..., i].flatten() for i, name in enumerate(estimands[0])}
series = pd.DataFrame(data)
series.describe().T

In [ ]:
series.quantile([0.05, 0.95]).T

In [ ]:
np.quantile(estimands[1][..., 2], 0.05), np.quantile(estimands[1][..., 2], 0.95)

In [ ]:
visualize_sampling_2d(
    samples.view(-1, 2)[n_samples // 2:],
    target_dist,
    samples[:150, :1, :]
)

## Hamiltonian Markov Chain Monte Carlo

https://colab.research.google.com/drive/1YQBSfS1Nb8a9TAMsV1RjWsiErWqXLbrj#scrollTo=VY3G25_m9HBZ
http://www.mcmchandbook.net/HandbookChapter5.pdf
https://towardsdatascience.com/python-hamiltonian-monte-carlo-from-scratch-955dba96a42d
https://arxiv.org/pdf/1701.02434.pdf

In [ ]:
mix = D.Categorical(torch.ones(2,))
comp = D.Independent(D.Normal(torch.Tensor([[0, 0],[4, 6]]), torch.Tensor([[1, 1], [1, 2]])), 1)
target_dist = D.MixtureSameFamily(mix, comp)

In [ ]:
target_dist.variance

In [ ]:
M = torch.Tensor([1/5, 1/10])  # Approximate inverse of covariance of target distribution
momentum_dist = D.Independent(D.Normal(torch.Tensor([0, 0]), M), 1)

In [ ]:
momentum_dist.batch_shape, momentum_dist.event_shape

In [ ]:
initial_point = torch.tensor([0.1, 5.0])
n_samples = 1000
leapfrog_steps = 10
leapfrog_stepsize = 0.1

In [ ]:
samples, accept_reject, full_hmc_path = hamiltonian_monte_carlo(initial_point, n_samples, momentum_dist, target_dist.log_prob,
                                                                leapfrog_steps=leapfrog_steps, leapfrog_stepsize=leapfrog_stepsize,
                                                                return_full_path=True)
torch.tensor(accept_reject).mean()

In [ ]:
full_hmc_path.shape

In [ ]:
visualize_sampling_2d(samples, target_dist, connect_samples=full_hmc_path[:15])

### Batched HMC

In [ ]:
mix = D.Categorical(torch.ones(2,))
comp = D.Independent(D.Normal(torch.Tensor([[0, 0],[4, 6]]), torch.Tensor([[1, 1], [1, 2]])), 1)
target_dist = D.MixtureSameFamily(mix, comp)

In [ ]:
target_dist.stddev

In [ ]:
n_chains = 8

In [ ]:
M = torch.Tensor([1/2, 1/3])  # Approximate inverse of covariance of target distribution
momentum_dist = D.Independent(D.Normal(torch.Tensor([0, 0]), M), 1)
momentum_dist.batch_shape, momentum_dist.event_shape

In [ ]:
initial_point = torch.tensor([0.1, 5]) + 1 * torch.randn(size=(n_chains, 2))
n_samples = 2000
leapfrog_steps = 10
leapfrog_stepsize = 0.1
initial_point.shape

In [ ]:
samples, chains, estimand_names, estimands, full_hmc_path = hamiltonian_monte_carlo_batched(initial_point, n_samples, momentum_dist, target_dist.log_prob,
                                                                                            leapfrog_steps=leapfrog_steps, leapfrog_stepsize=leapfrog_stepsize,
                                                                                            return_full_path=True, use_progressbar=True)
samples.shape, estimands[1].shape, full_hmc_path.shape

In [ ]:
metrics, variogram, autocorrelations, T = convergence_test(estimand_names, estimands[n_samples // 2:])
metrics

In [ ]:
create_estimates_report(estimand_names, estimands)

In [ ]:
fig, ax = plot_autocorrelations(estimand_names, autocorrelations, T)

In [ ]:
visualize_sampling_2d(
    chains[n_samples//2:].view(-1, 2)[:1000],
    target_dist,
    full_hmc_path[:10, :, 0, :]  # All leapfrog steps for 15 samples in chain 0
)

# Assessing convergence

### Can we batch the gradient computation?
Examination done before imlpementation of batched HMC

In [ ]:
# First, compute the gradient independently for two proposals

In [ ]:
mix = D.Categorical(torch.ones(2,))
comp = D.Independent(D.Normal(torch.Tensor([[0, 0],[4, 6]]), torch.Tensor([[1, 1], [1, 2]])), 1)
target_dist = D.MixtureSameFamily(mix, comp)

In [ ]:
proposal_sample = torch.tensor([1.0, 2.0])

In [ ]:
proposal_sample.requires_grad = True
proposal_sample.grad = None
target_dist.log_prob(proposal_sample).backward()
proposal_sample.requires_grad = False
proposal_sample.grad

In [ ]:
proposal_sample = torch.tensor([2.0, 4.0])

In [ ]:
proposal_sample.requires_grad_(True)
proposal_sample.grad = None
target_dist.log_prob(proposal_sample).backward()
proposal_sample.requires_grad_(False)
proposal_sample.grad

In [ ]:
# Now, compute the gradient via batching by exploting summation in the `log_prob`

In [ ]:
mix = D.Categorical(torch.ones(2,))
comp = D.Independent(D.Normal(torch.Tensor([[0, 0],[4, 6]]), torch.Tensor([[1, 1], [1, 2]])), 1)
target_dist = D.MixtureSameFamily(mix, comp)

In [ ]:
proposal_sample = torch.tensor([[1.0, 2.0], [2.0, 4.0]])

In [ ]:
proposal_sample.requires_grad = True
proposal_sample.grad = None
target_dist.log_prob(proposal_sample).sum().backward()
proposal_sample.requires_grad = False

In [ ]:
proposal_sample.grad

## First nonnegative element in tensor

In [ ]:
x = torch.randn(5, 8)
x[:, 2] = -torch.ones(5)
x

In [ ]:
nonnegative = (x > 0)
nonnegative

In [ ]:
cumsum = nonnegative.cumsum(axis=0)
cumsum

In [ ]:
first_nonzero_bool_map = (nonnegative.cumsum(0) == 1) & nonnegative
first_nonzero_bool_map

In [ ]:
naive_nonzero = ((nonnegative.cumsum(0) == 1) & nonnegative).max(0).indices
naive_nonzero

In [ ]:
all_nonzero = cumsum[-1] == 0
all_nonzero

In [ ]:
naive_nonzero[all_nonzero] = -1
naive_nonzero

In [ ]:
first_nonzero_bool_map

In [ ]:
x

In [ ]:
cumsum > 0

In [ ]:
idx = cumsum <= 0 
x[~idx] = 0
x.sum(axis=0)

In [ ]:
x[idx]

In [ ]:
index = (torch.ones(8) * 4)
index = index.type(torch.LongTensor)

In [ ]:
index.dtype, index

In [ ]:
x

In [ ]:
idx = torch.LongTensor([1, 2, 3, 4, 0, 1, 2, 3])
idx

In [ ]:
x[idx]

In [ ]:
idx = torch.LongTensor([1, 2, 3, 4, 0])

In [ ]:
torch.index_select(x, 0, idx, out=None)